가상환경 : 랭체인 커뮤니티 깔려 있는 RAG_ENV 환경으로 진행^^

In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA

In [36]:
import sys
print(sys.executable)

/home/user1/miniconda3/envs/rag_env/bin/python


In [37]:
import numpy, langchain, fitz, faiss, pinecone_text
print("NumPy:", numpy.__version__)
print("LangChain:", langchain.__version__)

NumPy: 1.26.4
LangChain: 0.2.17


In [38]:
import torch
import faiss

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"FAISS GPU support: {faiss.get_num_gpus()}")

CUDA available: True
GPU count: 1
FAISS GPU support: 0


In [39]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("pdfs/1-s2.0-S0304383525004598-main.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 2


In [35]:
# print(docs[10].page_content)

In [21]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 5


In [22]:
# 단계 3: 임베딩(Embedding) 생성
# embeddings = OpenAIEmbeddings()
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"  # 기본 Ollama 서버 주소
)

In [23]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [39]:
# for doc in vectorstore.similarity_search("구글"):
#     print(doc.page_content)

In [24]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [25]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("주된 신약명은 무엇인가요?")

[Document(metadata={'source': 'pdfs/1-s2.0-S0304383525004598-main.pdf', 'file_path': 'pdfs/1-s2.0-S0304383525004598-main.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.7', 'title': 'Corrigendum to “Combined inhibition of CDK4/6 and AKT is highly effective against the luminal androgen receptor (LAR) subtype of triple negative breast cancer” [604 (2024) 217219 1–12]', 'author': 'María Rosario Chica-Parrado', 'subject': 'Cancer Letters, 630 (2025) 217891. doi:10.1016/j.canlet.2025.217891', 'keywords': '', 'creator': 'Elsevier', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationDate': 'D:20250906105809Z', 'modDate': 'D:20250906110021Z', 'trapped': ''}, page_content='a UT Southwestern Simmons Comprehensive Cancer Center, Dallas, TX, 75390, USA\nb Yonsei University College of Medicine, Seoul, South Korea\nc Dept. of Life Science, Hanyang University, Seoul, South Korea\nThe authors regret to report an error in our recently published article \nby Chica-Parrado et al. titled “Combi

In [42]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
# prompt = PromptTemplate.from_template(
#     """You are an assistant for question-answering tasks. 
# Use the following pieces of retrieved context to answer the question. 
# If you don't know the answer, just say that you don't know. 
# Answer in Korean.

# #Context: 
# {context}

# #Question:
# {question}

# #Answer:"""
# )

In [29]:
from langchain.llms.base import LLM
from typing import Optional, List, Any
import subprocess

class OllamaLLM(LLM):
    model_name: str = "gpt-oss"
    temperature: float = 0.0

    def _call(
        self, 
        prompt: str, 
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None
    ) -> str:
        result = subprocess.run(
            ['ollama', 'run', self.model_name, prompt],
            capture_output=True, 
            text=True
        )
        
        if result.returncode != 0:
            raise RuntimeError(f"Ollama LLM failed: {result.stderr}")
        return result.stdout.strip()

    @property
    def _identifying_params(self):
        return {"model_name": self.model_name, "temperature": self.temperature}

    @property
    def _llm_type(self):
        return "ollama"

In [30]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = OllamaLLM(model_name="gpt-oss", temperature=0)

In [31]:
# 단계 8: 체인(Chain) 생성
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # 또는 "map_reduce", "refine", "map_rerank"
)

오류 코드

In [32]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.

# 방법 2: __call__ 사용 (더 간단)
question = "논문에서 주된 신약명은?"
response = chain({"query": question})
print(response)

KeyboardInterrupt: 

ollama api 방식으로 변경

In [33]:
import requests
from typing import Optional, List, Any
from langchain.llms.base import LLM

class OllamaLLM(LLM):
    model_name: str = "gpt-oss"
    base_url: str = "http://localhost:11434"
    timeout: int = 300  # 13GB 모델이라 넉넉히 설정
    
    def _call(
        self, 
        prompt: str, 
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None
    ) -> str:
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }
        
        try:
            response = requests.post(url, json=payload, timeout=self.timeout)
            response.raise_for_status()
            return response.json()['response']
        except requests.Timeout:
            raise RuntimeError(f"Ollama timed out after {self.timeout}s")
        except Exception as e:
            raise RuntimeError(f"Ollama API error: {str(e)}")
    
    @property
    def _llm_type(self) -> str:
        return "ollama"

# 사용
llm = OllamaLLM(model_name="gpt-oss")

단계별 디버깅

In [34]:
# Step 1: Ollama 모델 직접 테스트
from subprocess import run

result = run(['ollama', 'run', 'gpt-oss', 'Hello, test'], 
             capture_output=True, text=True, timeout=30)
print("stdout:", result.stdout)
print("stderr:", result.stderr)
print("returncode:", result.returncode)

TimeoutExpired: Command '['ollama', 'run', 'gpt-oss', 'Hello, test']' timed out after 30 seconds

In [27]:
# 1. LLM 직접 테스트
llm = OllamaLLM(model_name="gpt-oss")
print(llm("What is 2+2?"))

# 2. 체인 실행
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    verbose=True
)

response = chain({"query": "논문에서 주된 신약명은?"})
print(response['result'])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


4

> Finished chain.
I don't know.


텍스트 모델 버전 프롬프트

In [19]:
from langchain.prompts import PromptTemplate

# 커스텀 프롬프트 정의
prompt_template = """You are a biomedical text analysis assistant.

==== Document Excerpt Start ====
{context}
==== Document Excerpt End ====

Task:
- Extract only **generic (non-brand)** drug names mentioned in the 'Results' section of the provided research paper.
- If no drug names are found in the Results section, then extract them from the 'Methods' section instead.
- Exclude any drug names appearing in the 'References' section.
- Exclude gene names, protein names, biomarkers, molecular targets, and signaling pathway components (e.g., EGFR, MEK, CDK4, PTEN, ERBB2, FGFR3, etc.).
- Exclude experimental or investigational code names (e.g., TAS0728, T-DM1, AZD9496, MK-2206, LY294002).
- Include only drugs that have an established or officially recognized generic name (INN).
- Extract only therapeutic agents or compounds actually tested, administered, or mentioned as drugs.
- List only essential and unique generic drug names.

Output format:
List all unique generic drug names separated by semicolons (;), without quotation marks.
Do not include any additional text, explanation, or numbering.

Example output:
nivolumab; pembrolizumab; sunitinib; fulvestrant

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# RetrievalQA에 적용
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=True
)

# 실행
question = "Extract generic drug names from this paper"
response = chain({"query": question})
print(response['result'])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
sotorasib; adavosertib; puromycin


멀티모달 버전 프롬프트